In [1]:
!python setup.py build_ext --inplace

running build_ext
running build_ext


In [2]:
import network_simulation as net_sims
import popconstruct_hyperdirect as pops
import plotting_helper_functions as plt_help
import pipeline_creation as pl_creat
import pandas as pd
import os
import time
import pdb

import pylab as pl
import pickle
import numpy as np
import pandas as pd
import pingouin
import sklearn 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

data_dir = "./Data/weights_analysis/lesion-experiments/GPeA-iSPN/"
figure_dir = "./Figures/weights_analysis/lesion-experiments/"

2023-11-22 10:24:02,776	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2023-11-22 10:24:02,784	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


In [3]:
experimentchoice = 'stopsignal'
number_of_choices = 1
#pl_creat.choose_pipeline(experimentchoice)
#pl = pl_creat.create_main_pipeline(runloop=True)

In [5]:
environment = {
    
    'experimentchoice': experimentchoice,
    'inter_trial_interval': None,
    'thalamic_threshold': 30.,
    'movement_time': ['constant', 300], #['constant', 300], #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout': 300, #default 1000
    
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : pd.DataFrame([["left"]], columns=['action']), #action channels related parameters (init_params.py)
    'number_of_choices':number_of_choices,
    #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df_set': pd.DataFrame([[0.5]],columns=["left"]), #pd.DataFrame([[0.3,0.7]],columns=['left','right']), #initialized Q-values df (qvalues.py)  
    'n_trials': 50, #number of trials (generateepochs.py)
    'volatility': [None,"exact"], #frequency of changepoints (generateepochs.py)
    'conflict': (1.0), #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
    'maxstim': 1.15, # amplitude of the cortical input over base line #0.85
    'sustainedfraction': 0.75,
    
    #Stop 1
    'stop_signal_present':False,
    'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_signal_amplitude': 0.6, # amplitude of the stop signal over base line
    'stop_signal_onset': 70., #in ms #antes 200
    'stop_signal_duration' : 145., 
    'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signa
    'stop_signal_population':["STNE"],
    
    #Stop 2
    'stop_2_signal_present': True,
    'stop_2_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_2_signal_amplitude': 0.4, # amplitude of the stop signal over base line
    'stop_2_signal_onset': 70., #in ms #antes 200
    'stop_2_signal_duration' : 145.,
    'stop_2_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signal
    'stop_2_signal_population':["GPeA"],                           
    
    
    'decision_channel' : 'all', #'all'(all channels)
                              # '0' : left channel considered 
                              # '1' : right channel considered
    
    'record_variables': [], #["weight","optogenetic_input", "stop_input_1", "stop_input_2"],
    
    #Opto
    'opt_signal_present': True,
    'opt_signal_probability': 1., #[0,1,2,3,4], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude': 0.4, # ampitude of the stop signal over base line
    'opt_signal_onset': 70., # in ms
    'opt_signal_duration': 145.,
    'opt_signal_channel': "all", # "all" (all channels are given the stop signal) 
    'opt_signal_population':["D2STR"],    
}

In [6]:
seeds = 0 #np.random.randint(0,9999999)
print(seeds)
offset = 0 #To start from last sim run (remember number of last sim) 

0


In [7]:
connections =  pops.helper_poppathways(None,None)

channels: one
scaling_conn 1
scaling_wts 2.0


In [9]:
connections.loc[connections['src'] == 'GPeA'][['src', 'dest', 'receptor', 'type', 'con', 'eff', 'plastic']]

,src,dest,receptor,type,con,eff,plastic
23,GPeA`,FSI`,GABA`,all`,0.4`,0.02`,False`
24,GPeA`,D2STR`,GABA`,syn`,0.4`,0.12`,False`
25,GPeA`,D1STR`,GABA`,syn`,0.4`,0.32`,False`
26,GPeA`,GPeA`,GABA`,all`,0.4`,0.3`,False`


### Double

In [ ]:
weak_factor_1 = 2/3 #0.0433
weak_factor_2 = 1/3 #0.0217
solutions_AMPA = [np.round(0.1*weak_factor_2, 4), np.round(0.1*weak_factor_1, 4), 
                  0.0817, 0.0967, 0.0987, 0.10]
solutions_NMDA = [np.round(1.51*weak_factor_2, 4), np.round(1.51*weak_factor_1,4), 
                  1.2067, 1.4067, 1.4567, 1.51] 
params_to_fit = [['STNE', 'GPeP', 'AMPA', 'syn', 0.161666, 0.1, False], 
                 ['STNE', 'GPeP', 'NMDA', 'syn', 0.161666, 1.51, False]] 
print(solutions_AMPA)
print(solutions_NMDA)

In [ ]:
for i in np.arange(len(solutions_AMPA)):
    if i < offset:
        continue
    print("i",i)
                 
    new_params =  connections.copy() 
    #new_params = pd.DataFrame(columns=['src', 'dest', 'receptor', 'type', 'con', 'eff'])
    
    sol_AMPA = solutions_AMPA[i]  
    sol_NMDA = solutions_NMDA[i] 
   
    for i1,k in enumerate(params_to_fit):
        
        ind = new_params[(new_params["src"] == k[0])&(new_params["dest"] == k[1])&(new_params["receptor"] == k[2])&(new_params["type"] == k[3])&(new_params["con"] == k[4])].index.tolist()
        #print(ind)
        
        if new_params.loc[ind[0], "receptor"] == 'AMPA':
        
            val = sol_AMPA
            print('weight AMPA:', val)
            ind = new_params[(new_params["src"] == k[0])&(new_params["dest"] == k[1])&(new_params["receptor"] == k[2])&(new_params["type"] == k[3])&(new_params["con"] == k[4])].index.tolist()
            new_params.loc[ind[0], "eff"] = val
        
        elif new_params.loc[ind[0], "receptor"] == 'NMDA':
            
            val = sol_NMDA
            print('weight GABA:', val)
            ind = new_params[(new_params["src"] == k[0])&(new_params["dest"] == k[1])&(new_params["receptor"] == k[2])&(new_params["type"] == k[3])&(new_params["con"] == k[4])].index.tolist()
            new_params.loc[ind[0], "eff"] = val
        
        else:
            print('Error')

    new_env = environment.copy()
    new_env['newpathways'] = new_params
 
    seed = seeds+i
    
    results = net_sims.run_simulation(seed,new_env)

### Single

In [10]:
#weak_factor_1 = 2/3 #0.0433
#weak_factor_2 = 1/3 #0.0217
#solutions = [0.28, 0.22, 0.18, 0.12, 0.09, 0]
solutions = [0.12, 0.08, 0.04, 0.02, 0]
params_to_fit = [['GPeA', 'D2STR', 'GABA', 'syn', 0.4, 0.12, False]] 
print(solutions)

[0.12, 0.08, 0.04, 0.02, 0]


In [11]:
for i in np.arange(len(solutions)):
    if i < offset:
        continue
    print("i",i)
                 
    new_params =  connections.copy() 
    #new_params = pd.DataFrame(columns=['src', 'dest', 'receptor', 'type', 'con', 'eff'])
    sol = solutions[i]   
   
    for i1,k in enumerate(params_to_fit):
        
        val = sol
        print('weight:', val)
        ind = new_params[(new_params["src"] == k[0])&(new_params["dest"] == k[1])&(new_params["receptor"] == k[2])&(new_params["type"] == k[3])&(new_params["con"] == k[4])].index.tolist()
        new_params.loc[ind[0], "eff"] = val


    new_env = environment.copy()
    new_env['newpathways'] = new_params
 
    seed = seeds+i
    
    results = net_sims.run_simulation(seed,new_env)

i 0
weight: 0.12
{'experimentchoice': 'stopsignal', 'inter_trial_interval': None, 'thalamic_threshold': 30.0, 'movement_time': ['constant', 300], 'choice_timeout': 300, 'params': None, 'pops': None, 'receps': None, 'base': None, 'dpmns': None, 'd1': None, 'd2': None, 'channels':   action
0   left, 'number_of_choices': 1, 'newpathways':        src    dest receptor  type        con      eff plastic  \
0     LIP`  D1STR`    AMPA`  syn`       1.0`   0.022`   True`   
1     LIP`  D1STR`    NMDA`  syn`       1.0`    0.03`  False`   
2     LIP`  D2STR`    AMPA`  syn`       1.0`   0.022`   True`   
3     LIP`  D2STR`    NMDA`  syn`       1.0`   0.028`  False`   
4     LIP`    FSI`    AMPA`  all`       1.0`    0.17`  False`   
5     LIP`     Th`    AMPA`  syn`       1.0`   0.025`  False`   
6     LIP`     Th`    NMDA`  syn`       1.0`   0.029`  False`   
7   D1STR`  D1STR`    GABA`  syn`      0.45`    0.28`  False`   
8   D1STR`  D2STR`    GABA`  syn`      0.45`    0.28`  False`   
9   D1STR`  

(raylet) E1122 10:25:56.090834000 8642171584 chttp2_server.cc:1050]             {"created":"@1700645156.090483000","description":"No address added out of total 1 resolved","file":"external/com_github_grpc_grpc/src/core/ext/transport/chttp2/server/chttp2_server.cc","file_line":947,"referenced_errors":[{"created":"@1700645156.090179000","description":"Unable to configure socket","fd":28,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/tcp_server_utils_posix_common.cc","file_line":218,"referenced_errors":[{"created":"@1700645156.090171000","description":"Address already in use","errno":48,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/tcp_server_utils_posix_common.cc","file_line":191,"os_error":"Address already in use","syscall":"bind"}]}]}
(raylet) [2023-11-22 10:25:56,103 C 3842 24546] grpc_server.cc:110:  Check failed: server_ Failed to start the grpc server. The specified port is 10007. This means that Ray's core components will not be able to function correctly. If 

(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3837) in choose pipeline
(worker pid=3838) in choose pipeline
(worker pid=3838) in choose pipeline
(worker pid=3845) in choose pipeline
(worker pid=3845) in choose pipeline
(worker pid=3839) in choose pipeline
(worker pid=3839) in choose pipeline
(worker pid=3851) in choose pipeline
(worker pid=3850) in choose pipeline
(worker pid=3851) in choose pipeline
(worker pid=3850) in choose pipeline
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea426e3d0>
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea426e3d0>
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea426e3d0>
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea426e3d0>
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea426e3d0>
SpawnThreadMan

(worker pid=3845) channels: one
(worker pid=3845) scaling_conn 1
(worker pid=3845) scaling_wts 2.0
(worker pid=3845) channels: one
(worker pid=3845) scaling_conn 1
(worker pid=3845) scaling_wts 2.0
SpawnThreadManagers <backend.Pipeline object at 0x7f9e6804afd0>
(worker pid=3837) [18 31  5 27 10 12 29  7  0 44 47  8  9 37 41 38 33 22 48 16 45 14  1 32
(worker pid=3837)  42 46 15 28 49 23 35 40 25 39 20 21 34  6 26 17 36 43 11 13  3 19  2 30
(worker pid=3837)  24  4]
(worker pid=3838) [36 12 48 23 17 39 20 10 29 18  7  5  6  9 15 28 35  0 41  1 47 49 45 37
(worker pid=3838)  21 27 16 24 33 42 25 46 22 11 38  4 32 14  3  2 26 44 31 19  8 40 13 43
(worker pid=3838)  34 30]
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea42bd070>
(worker pid=3837) [18 31  5 27 10 12 29  7  0 44 47  8  9 37 41 38 33 22 48 16 45 14  1 32
(worker pid=3837)  42 46 15 28 49 23 35 40 25 39 20 21 34  6 26 17 36 43 11 13  3 19  2 30
(worker pid=3837)  24  4]
(worker pid=3838) [36 12 48 23 17 39 20 10 29 18  

SpawnThreadManagers <backend.Pipeline object at 0x7f9ea42c19d0>
(worker pid=3837) channels: one
(worker pid=3837) scaling_conn 1
(worker pid=3837) scaling_wts 2.0
(worker pid=3837) channels: one
(worker pid=3837) scaling_conn 1
(worker pid=3837) scaling_wts 2.0
SpawnThreadManagers <backend.Pipeline object at 0x7f9ea42c19d0>
(worker pid=3866) channels: one
(worker pid=3866) scaling_conn 1
(worker pid=3866) scaling_wts 2.0
(worker pid=3866) channels: one
(worker pid=3866) scaling_conn 1
(worker pid=3866) scaling_wts 2.0
(worker pid=3837) channels: one
(worker pid=3837) scaling_conn 1
(worker pid=3837) scaling_wts 2.0
(worker pid=3866) channels: one
(worker pid=3866) scaling_conn 1
(worker pid=3866) scaling_wts 2.0
(worker pid=3837) channels: one
(worker pid=3837) scaling_conn 1
(worker pid=3837) scaling_wts 2.0
(worker pid=3866) channels: one
(worker pid=3866) scaling_conn 1
(worker pid=3866) scaling_wts 2.0
(worker pid=3876) channels: one
(worker pid=3876) scaling_conn 1
(worker pid=387

...

(worker pid=3863) gateFRs [23.55555556]
(worker pid=3863) thresholds_crossed []
(worker pid=3863) gateFRs [23.55555556]
(worker pid=3863) thresholds_crossed []
(worker pid=3838) trial_num 35
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3838) trial_num 35
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3837) gateFRs [30.22222222]
(worker pid=3837) thresholds_crossed [0]
(worker pid=3837) gateFRs [30.22222222]
(worker pid=3837) thresholds_crossed [0]
(worker pid=3863) trial_num 33
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3863) trial_num 33
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3837) trial_num 33
(worker pid=3837)   action
(worker pid=3837) 0   left
(worker pid=3837) chosen_action: left
(worker pid=3837) trial_num 33
(worker pid=3837)   action
(work

(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3838) gateFRs [21.11111111]
(worker pid=3838) thresholds_crossed []
(worker pid=3838) gateFRs [21.11111111]
(worker pid=3838) thresholds_crossed []
(worker pid=3838) trial_num 39
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3838) trial_num 39
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3863) stop_2 stim started
(worker pid=3863) opt stim started
(worker pid=3863) stop_2 stim started
(worker pid=3863) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3863) gateFRs [22.88888889]
(worker pid=3863) thresholds_crossed []
(worker pid=3863) gateFRs [22.88888889]
(worker pid=3863) thresholds_crossed

(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) gateFRs [30.66666667]
(worker pid=3837) thresholds_crossed [0]
(worker pid=3837) gateFRs [30.66666667]
(worker pid=3837) thresholds_crossed [0]
(worker pid=3838) gateFRs [23.33333333]
(worker pid=3838) thresholds_crossed []
(worker pid=3838) gateFRs [23.33333333]
(worker pid=3838) thresholds_crossed []
(worker pid=3837) trial_num 40
(worker pid=3837)   action
(worker pid=3837) 0   left
(worker pid=3837) chosen_action: left
(worker pid=3837) trial_num 40
(worker pid=3837)   action
(worker pid=3837) 0   left
(worker pid=3837) chosen_action: left
(worker pid=3838) trial_num 43
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3838)

(worker pid=3863) gateFRs [20.22222222]
(worker pid=3863) thresholds_crossed []
(worker pid=3863) gateFRs [20.22222222]
(worker pid=3863) thresholds_crossed []
(worker pid=3838) trial_num 46
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3838) trial_num 46
(worker pid=3838)   action
(worker pid=3838) 0   left
(worker pid=3838) chosen_action: stop
(worker pid=3863) trial_num 44
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3863) trial_num 44
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3838) stop_2 stim started
(worker pid=3838) opt stim started
(worker pid=3837) gateFRs [24.22222222]
(worker pid=3837

(worker pid=3837) trial_num 47
(worker pid=3837)   action
(worker pid=3837) 0   left
(worker pid=3837) chosen_action: stop
(worker pid=3837) trial_num 47
(worker pid=3837)   action
(worker pid=3837) 0   left
(worker pid=3837) chosen_action: stop
(worker pid=3863) gateFRs [25.55555556]
(worker pid=3863) thresholds_crossed []
(worker pid=3863) gateFRs [25.55555556]
(worker pid=3863) thresholds_crossed []
(worker pid=3863) trial_num 48
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3863) trial_num 48
(worker pid=3863)   action
(worker pid=3863) 0   left
(worker pid=3863) chosen_action: stop
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) stop_2 stim started
(worker pid=3837) opt stim started
(worker pid=3837) gateFRs [24.44444444]
(worker pid=3837) thresholds_crossed []
(worker pid=3837) gateFRs [24.44444444]
(worker pid=3837) thresholds_crossed []
(worker pid=3863) stop_2 stim started
(wor